In [ ]:
!gshell init

In [ ]:
!gshell info --with-id 17axH-dYh-Wss_ec92ugSl8AmSuZS-xn9

In [ ]:
import signal
from pathlib import Path
import wget

parser_version = '1.6.7'
url = f'https://github.com/nemoware/document-parser/releases/download/{parser_version}/document-parser-{parser_version}.jar'
if not Path(f'document-parser-{parser_version}.jar').is_file():
    wget.download(url)

In [ ]:
import glob
import zipfile

with zipfile.ZipFile(f'./{glob.glob("ДД по практикам*.zip")[0]}',
                     'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
key_value = ['о нижеследующем:', 'нижеследующем:', 'о нижеследующем', 'нижеследующем']
for x in key_value[5:]:
    print(123)


In [ ]:
parser_version = '1.6.7'
!java -cp "document-parser-$parser_version/classes;document-parser-$parser_version/lib/*" com.nemo.document.parser.App -i "Документы/ДД по практикам/Практика поддержки региональных, розничных продаж и клиентского сервиса/Приложение Договор 6.docx"

In [ ]:
import subprocess

s = [
    "gshell",
    "init"
]

subprocess.run(s, stdin=subprocess.PIPE, stdout=subprocess.PIPE)

# Start

## Import

In [1]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import platform
import sys
import os
import json
import subprocess
import pandas as pd
import fnmatch
import base64
import requests
import time
import re
import numpy as np
import importlib
# from search_text import list_of_sheets, find_text
import search_text

## Create list of all docs path

In [2]:
arrOfAllDocs = []

for root, dirnames, filenames in os.walk('Документы'):
    if len(root.split('\\')) == 2: continue
    flag = False
    for i in root.split('\\'):
        if str(i).startswith('Исключена'):
            flag = True
            break
    if flag: continue
    for filename in fnmatch.filter(filenames, '*.docx'):
        arrOfAllDocs.append(os.path.join(root, filename))
    for filename in fnmatch.filter(filenames, '*.doc'):
        arrOfAllDocs.append(os.path.join(root, filename))


In [3]:
arrOfAllDocs[:]

['Документы\\Входящие по практикам\\Практика комм. логистики\\01-Шаблон Исковое заявление Типовой договор поставки ГПН (штраф за простой вагонов).docx',
 'Документы\\Входящие по практикам\\Практика комм. логистики\\Акт общей формы_ШАБЛОН_ЛЕД_дополнен.docx',
 'Документы\\Входящие по практикам\\Практика комм. логистики\\АКТ ОТБОРА ПРОБ_ШАБЛОН.docx',
 'Документы\\Входящие по практикам\\Практика комм. логистики\\АКТ ПРИЕМКИ по КАЧЕСТВУ_ШАБЛОН.docx',
 'Документы\\Входящие по практикам\\Практика комм. логистики\\Заявление ССП_шаблон на бланке ПАО.docx',
 'Документы\\Входящие по практикам\\Практика комм. логистики\\ПРЕТЕНЗИЯ_излишки ГПН - ГПН-РП_ШАБЛОН для Регламента.docx',
 'Документы\\Входящие по практикам\\Практика комм. логистики\\ПРЕТЕНЗИЯ_недостача ГПН - ГПН-РП_ШАБЛОН для Регламента.docx',
 'Документы\\Входящие по практикам\\Практика комм. логистики\\расчет претензии ГПН-Л к РЖД_ШАБЛОН.docx',
 'Документы\\Входящие по практикам\\Практика комм. логистики\\Соглашение о зачете встречных одн

In [8]:
importlib.reload(search_text)

index = 1
number_of_docs = 0
parser_version = '1.6.7'
root = "ДД по практикам"

with_neuron = False

tokenizer_path = "sberbank-ai/ruRoberta-large"
path_to_model = "./doc-classification/"

labels = ['Практика коммерческой логистики',
          'Практика недропользования и экологии',
          'Практика поддержки региональных, розничных продаж и клиентского сервиса',
          'Практика правового сопровождения закупок МТР и услуг общего профиля',
          'Практика правового сопровождения земельных отношений и сделок с недвижимым имуществом',
          'Практика правового сопровождения операционной деятельности БРД',
          'Практика правового сопровождения переработки и инфраструктуры',
          'Практика правовой поддержки брендов',
          'Практика правовой поддержки использования и коммерциализации ИС',
          'Практика правовой поддержки создания и приобретения ИС',
          'Практика промышленной безопасности и охраны труда',
          'Практика финансового и конкурентного права',
          'Практика экспорта, оптовых продаж и сбыта бизнес-единиц (БЕ)']

result = []
result_of_fail = []
result_of_possible = []
result_of_possible2 = []

s = [
    "java",
    "-jar",
    f"document-parser-{parser_version}.jar",
    "--server.port=8083"
]
headers = {
    'Content-type': 'application/json',
    'Accept': 'application/json; text/plain'
}

print(
    "Запуск document-parser на 8083 порту, если что-то пойдет не так, то руками УБЕЙТЕ java процесс"
)
java_subprocess = subprocess.Popen(s, creationflags=subprocess.CREATE_NEW_PROCESS_GROUP,
                                   stdout=subprocess.PIPE, encoding="utf-8")


def get_tokens(text):
    result = tokenizer(text, truncation=True, max_length=512)
    return result


def predicate_result(text):
    tokens = get_tokens(text)
    predictions = model.predict([tokens['input_ids']])['logits']
    predictions = tf.nn.softmax(predictions, name=None)[0].numpy()
    print(predictions)
    print(sum(predictions))

    return predictions


def is_true(text, path):
    if not with_neuron: return False
    result = predicate_result(text)
    argmax = np.argmax(result, axis=0)
    required_label = labels[argmax]
    if required_label in path:
        return True

    return False


time.sleep(2)
i = 1
while True:
    time.sleep(0.1)
    output_log_spring = java_subprocess.stdout.readline()
    sys.stdout.write("\rПроверка соединения #%i" % i)
    sys.stdout.flush()
    i += 1
    if output_log_spring.find("Started DocumentParserService") != -1:
        print("\nГотово")
        java_subprocess.stdout.close()
        break

print("Запустился успешно")
print("Общее количество документов =", len(arrOfAllDocs))

tokenizer = None
model = None

if with_neuron:
    tokenizer = AutoTokenizer.from_pretrained(str(tokenizer_path))
    model = TFAutoModelForSequenceClassification.from_pretrained(
        str(path_to_model), num_labels=len(labels), from_pt=False
    )

for docs in arrOfAllDocs:
    try:
        file = open(docs, 'rb')
        encoded_string = base64.b64encode(file.read())
        encoded_string = str(encoded_string)[2:-1]
    except Exception as e:
        print(f"\nОшибка в файле {docs}")
        print(f"при конвертации в base64, исключение = {e.msg}")
        print("=" * 200)
        continue
    is_doc = True
    is_docx = True
    is_not_bad_doc = True
    doc_type = docs.split(".")[-1].upper()
    while is_doc or is_docx or is_not_bad_doc:
        response = requests.post(
            "http://localhost:8083/document-parser",
            data=json.dumps({
                "base64Content": encoded_string,
                "documentFileType": doc_type
            }),
            headers=headers
        )
        resArr = []
        if 'message' in response.json() and not (is_doc == False and is_docx == False):
            print(response.json())
            if doc_type == 'DOC':
                is_doc = False
                doc_type = 'DOCX'
                continue
            if doc_type == 'DOCX':
                is_docx = False
                doc_type = 'DOC'
                continue

        try:
            resArr = response.json()['documents']
        except Exception as e:
            print(f"\nОшибка в файле {docs}")
            print(f"Ответ от парсера {response.json()}")
            print("=" * 200)
        finally:
            is_doc = False
            is_docx = False
            is_not_bad_doc = False

    sys.stdout.write(f"\rПроверка документа под номером {index} и ИД {len(result)}")
    sys.stdout.flush()
    index += 1

    document = []
    if resArr:
        document = resArr[0]
    else:
        continue

    result_of_single_doc, enum = search_text.find_text(document, path=docs)
    if enum == search_text.list_of_sheets.GOOD:
        result.append(result_of_single_doc)
    if enum == search_text.list_of_sheets.BAD:
        result_of_fail.append(result_of_single_doc)
    if enum == search_text.list_of_sheets.TEST:
        result_of_possible.append(result_of_single_doc)
    if enum == search_text.list_of_sheets.TEST2:
        result_of_possible2.append(result_of_single_doc)

#Смерть java процессу!
if platform.system() == 'Windows':
    subprocess.run("TASKKILL /F /PID {pid} /T".format(pid=java_subprocess.pid))
elif platform.system() == 'Linux':
    os.kill(java_subprocess.pid, signal.SIGTERM)
else:
    print('Не известная платформа, убейте в ручную процесс java')
# os.killpg(os.getpgid(java_subprocess.pid), signal.SIGTERM)

writer = pd.ExcelWriter("classifier.xlsx", engine='xlsxwriter')

df = pd.DataFrame(result)
df.to_excel(writer, 'good', engine='xlsxwriter')
sheets_good = writer.sheets['good']
sheets_good.autofilter(0, 0, df.shape[0], df.shape[1])

df = pd.DataFrame(result_of_fail)
df.to_excel(writer, 'SO SO', engine='xlsxwriter')
sheets_bad = writer.sheets['SO SO']
sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])

df = pd.DataFrame(result_of_possible)
df.to_excel(writer, 'Новые практики', engine='xlsxwriter')
sheets_bad = writer.sheets['Новые практики']
sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])

df = pd.DataFrame(result_of_possible2)
df.to_excel(writer, 'Результат по новым практикам', engine='xlsxwriter')
sheets_bad = writer.sheets['Результат по новым практикам']
sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])

writer.save()
print("\nФайл создан")

Запуск document-parser на 8083 порту, если что-то пойдет не так, то руками УБЕЙТЕ java процесс
Проверка соединения #31
Готово
Запустился успешно
Общее количество документов = 1972
Проверка документа под номером 277 и ИД 18{'timestamp': '2022-02-10T22:23:27.532+0000', 'status': 400, 'error': 'Bad Request', 'message': 'No valid entries or contents found, this is not a valid OOXML (Office Open XML) file', 'path': '/document-parser', 'version': '1.6.7'}
{'timestamp': '2022-02-10T22:23:27.544+0000', 'status': 400, 'error': 'Bad Request', 'message': 'The document is really a UNKNOWN file', 'path': '/document-parser', 'version': '1.6.7'}

Ошибка в файле Документы\ДД по практикам\Практика недропользования и экологии\~$Заказ.docx
Ответ от парсера {'timestamp': '2022-02-10T22:23:27.548+0000', 'status': 400, 'error': 'Bad Request', 'message': 'No valid entries or contents found, this is not a valid OOXML (Office Open XML) file', 'path': '/document-parser', 'version': '1.6.7'}
Проверка документа п